Semantic Analysis of IMDB moview review dataset.
A tutorial by Tensorflow

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88548
MAXLEN = 250
BATCH_SIZE = 64

(train_data,train_labels), (test_data,test_labels) = imdb.load_data(num_words=VOCAB_SIZE) 

we have to given review of same length to our neural network that's why we have to use padding
if reveiw is greater then 250 words trim it otherwise padd "0" to make it to 250 words

In [2]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [3]:
model = tf.keras.Sequential(
    [tf.keras.layers.Embedding(VOCAB_SIZE,32), # word embedding implementing vectors for our words
     tf.keras.layers.LSTM(32),
     tf.keras.layers.Dense(1, activation="sigmoid")
    ]
)


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2833536   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,841,889
Trainable params: 2,841,889
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
history = model.fit(train_data,train_labels, epochs=3, validation_split=0.2)

Epoch 1/3
625/625 [==============================] - 80s 122ms/step - loss: 0.1775 - accuracy: 0.9355 - val_loss: 0.4497 - val_accuracy: 0.8594
Epoch 2/3
625/625 [==============================] - 83s 133ms/step - loss: 0.1434 - accuracy: 0.9503 - val_loss: 0.3368 - val_accuracy: 0.8704
Epoch 3/3
625/625 [==============================] - 82s 132ms/step - loss: 0.1260 - accuracy: 0.9571 - val_loss: 0.3017 - val_accuracy: 0.8810


In [7]:
result = model.evaluate(test_data,test_labels)
print(result)

782/782 [==============================] - 32s 41ms/step - loss: 0.3671 - accuracy: 0.8560
[0.36713266372680664, 0.8559600114822388]


In [8]:
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 1s 1us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [9]:
reverse_word_index = {value:key for (key,value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num]+ " "
    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [18]:
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])

positive_review = "Oh I just love the movie. It was so amazing. I will definately watch it again"
predict(positive_review)

negative_review = "I don't like it"
predict(negative_review)

[0.7116824]
[0.5614304]
